In [1]:
import os
import json
from detectron2.data import MetadataCatalog, DatasetCatalog


from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/home/mina/ml_project/ML_project_core/src/coco_dental/annotations/instances_train2017.json", "/home/mina/ml_project/ML_project_core/src/coco_dental/train2017/")
register_coco_instances("my_dataset_val", {}, "/home/mina/ml_project/ML_project_core/src/coco_dental/annotations/instances_val2017.json", "/home/mina/ml_project/ML_project_core/src/coco_dental/val2017/")
# register_coco_instances("my_dataset_test", {}, "../detectron2/p10-v2/test/_annotations.coco.json", "../detectron2/p10-v2/test")

#MetadataCatalog.get("my_dataset_train").thing_classes = ["Eclosed egg", "Egg", "Larvae", "Lump"]
#MetadataCatalog.get("my_dataset_test").thing_classes = ["Eclosed egg", "Egg", "Larvae", "Lump"]

#visualize training data
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

In [2]:
import random
from detectron2.utils.visualizer import Visualizer

# Our trainer
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [3]:
from detectron2.config import get_cfg
from detectron2 import model_zoo


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 50000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 32  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.OUTPUT_DIR = "/home/mina/ml_project/ML_project_core/src/coco_dental/detectron2_mask_rcnn_No_flip_mask_rcnn_R_101_FPN_3x"
cfg.TEST.EVAL_PERIOD = 500

In [7]:
print(cfg.SOLVER.IMS_PER_BATCH)
!nvidia-smi

2
Sun Dec 11 14:43:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 4000     On   | 00000000:05:00.0 Off |                  N/A |
| 30%   35C    P8     3W / 125W |   4125MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----

In [4]:
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer

trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[12/11 02:28:10 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (33, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (33,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (128, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (128,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (32, 256, 1

[12/11 02:28:12 d2.engine.train_loop]: Starting training from iteration 0


/home/mina/ml_project/ML_project_core/src/detectron2/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  torch.stack([torch.from_numpy(np.ascontiguousarray(x)) for x in masks])
/home/mina/ml_project/ML_project_core/src/detectron2/detectron2/data/detection_utils.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before conver

[12/11 02:28:24 d2.utils.events]:  eta: 8:15:23  iter: 19  total_loss: 6.823  loss_cls: 3.274  loss_box_reg: 0.628  loss_mask: 0.693  loss_rpn_cls: 2.024  loss_rpn_loc: 0.2087  time: 0.5938  data_time: 0.0394  lr: 4.9953e-06  max_mem: 3457M
[12/11 02:28:36 d2.utils.events]:  eta: 8:17:50  iter: 39  total_loss: 5.343  loss_cls: 3.178  loss_box_reg: 0.7522  loss_mask: 0.6928  loss_rpn_cls: 0.5534  loss_rpn_loc: 0.1876  time: 0.5973  data_time: 0.0134  lr: 9.9902e-06  max_mem: 3457M
[12/11 02:28:48 d2.utils.events]:  eta: 8:21:03  iter: 59  total_loss: 4.681  loss_cls: 2.961  loss_box_reg: 0.7919  loss_mask: 0.6926  loss_rpn_cls: 0.08271  loss_rpn_loc: 0.1619  time: 0.5987  data_time: 0.0140  lr: 1.4985e-05  max_mem: 3457M
[12/11 02:29:00 d2.utils.events]:  eta: 8:23:23  iter: 79  total_loss: 4.342  loss_cls: 2.546  loss_box_reg: 0.8122  loss_mask: 0.6922  loss_rpn_cls: 0.07886  loss_rpn_loc: 0.1663  time: 0.6004  data_time: 0.0136  lr: 1.998e-05  max_mem: 3457M
[12/11 02:29:12 d2.utils.e